In [1]:
# Import necessary libraries
import pandas as pd
from pycaret.classification import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pycaret.classification import *

In [2]:
# Load and examine the dataset structure
data_train = pd.read_csv('dataset/titanic/train.csv')
display(data_train.info())
display(data_train.describe())
display(data_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Drop unnecessary columns
data_train.drop(['Name', 'Ticket', 'Embarked', 'Cabin', 'PassengerId'], axis=1, inplace=True)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data_train, random_state=42, train_size=0.8)

In [4]:
# Initialize the setup
clf1 = setup(data = data_train, 
             target = 'Survived', 
             categorical_features = ['Pclass','Sex','Age','Fare'])

,Description,Value
0,Session id,6229
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 7)"
4,Transformed data shape,"(891, 9)"
5,Transformed train set shape,"(623, 9)"
6,Transformed test set shape,"(268, 9)"
7,Numeric features,2
8,Categorical features,4
9,Rows with missing values,19.9%


In [5]:
# Compare models
compare_models(n_select=16)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7849,0.8371,0.6699,0.7501,0.7045,0.5367,0.5415,0.3520
ridge,Ridge Classifier,0.7833,0.8284,0.6868,0.7347,0.7067,0.5358,0.5391,0.0130
knn,K Neighbors Classifier,0.7784,0.7990,0.6822,0.7291,0.7017,0.5263,0.5300,0.0200
lda,Linear Discriminant Analysis,0.7543,0.8046,0.6699,0.6888,0.6772,0.4792,0.4810,0.0160
et,Extra Trees Classifier,0.7448,0.7923,0.6529,0.6791,0.6618,0.4576,0.4615,0.0380
rf,Random Forest Classifier,0.7383,0.7877,0.6529,0.6660,0.6563,0.4457,0.4483,0.0430
gbc,Gradient Boosting Classifier,0.7270,0.7490,0.6188,0.6570,0.6339,0.4169,0.4203,0.0260
lightgbm,Light Gradient Boosting Machine,0.7223,0.7364,0.6355,0.6433,0.6357,0.4120,0.4153,0.2090
nb,Naive Bayes,0.7190,0.8018,0.7241,0.6177,0.6641,0.4261,0.4327,0.0140
ada,Ada Boost Classifier,0.7125,0.7033,0.5940,0.6411,0.6132,0.3856,0.3887,0.0230


[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=6229, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, positive=False, random_state=6229, solver='auto',
                 tol=0.0001),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                      weights='uniform'),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
   

In [6]:
# 創建模型
model = create_model('rf')  # 以隨機森林為例

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6825,0.7687,0.5417,0.5909,0.5652,0.3160,0.3167
1,0.8571,0.8494,0.8333,0.8000,0.8163,0.6995,0.6999
2,0.7143,0.8114,0.6667,0.6154,0.6400,0.4038,0.4047
3,0.7742,0.7637,0.6957,0.6957,0.6957,0.5162,0.5162
4,0.7258,0.7341,0.6250,0.6522,0.6383,0.4177,0.4179
5,0.7903,0.8920,0.6250,0.7895,0.6977,0.5405,0.5491
6,0.6613,0.7840,0.7083,0.5484,0.6182,0.3226,0.3311
7,0.6774,0.7259,0.5000,0.6000,0.5455,0.2986,0.3016
8,0.7581,0.6919,0.6250,0.7143,0.6667,0.4781,0.4807


In [7]:
# 導入 Optuna
import optuna

ModuleNotFoundError: No module named 'optuna'

In [8]:
# 定義 Optuna 的目標函數
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    }
    model = create_model('rf', **params)
    tuned_model = tune_model(model)
    return tuned_model

In [9]:
# 創建一個研究並優化目標函數
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [10]:
# 獲取最佳試驗
best_trial = study.best_trial
print(f'Best trial: {best_trial.params}')

In [11]:
# 使用最佳參數創建並調優模型
best_params = best_trial.params
model = create_model('rf', **best_params)
tuned_model = tune_model(model)

In [12]:
# 評估調優後的模型
evaluate_model(tuned_model)

In [13]:
# 保存調優後的模型
save_model(tuned_model, 'best_tuned_model_optuna')